## Data Extraction and data wrangling workflow

### Logic / pipline

1. Purple Air 

1.1. Data is extracted manually as csv files and put on /data/purpleair/sample/

1.2. Data wrangling is done by script

More information in the [/data/purpleair/README.md](/data/purpleair/README.md)

purpleair/merge-pa.py script create first step of csv from the specified folder as below.

Two parameters to this script:

*csv*: csv filename to store the data after wrangling

*folder*: the folder where are located the purpleair csv files (one csv by groundstation) 

In [1]:
%run purpleair/merge-pa.py --csv bigtable.csv --folder purpleair/sample

2. Open AQ

2.1. Data was extracted stored into a s3 Amazon storage

2.2. Data wrangling to extract and merge more groundstations into CSV is done by script here as follow:

In [2]:
import pandas as pd
import numpy as np
import s3fs
import json

In [3]:
openAQcsv= 's3://openaqpm25dcarea/data_load_DC/Unsaved/2020/03/21/5f0f7603-af35-4673-b7cd-d1685d04fed7.csv'
df_openAQ = pd.read_csv(openAQcsv)


/home/julien/.local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (12) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


2.2.3. parsing date and coordinates column to extract X, Y and UTC datetime

date format is string with json structure as : '{utc=2017-08-11T05:00:00.000Z, local=2017-08-11T00:00:00-05:00}'

coordinates format is string with a json tructure as : '{latitude=38.92185, longitude=-77.013176}'

To be json compliant = sings should be replaced by : signs
Also missing double quote for strings

In [4]:
df_PM25 = df_openAQ[df_openAQ["parameter"] == 'pm25']
len(df_PM25)
df_PM25.head()

,date,parameter,location,value,unit,city,attribution,averagingperiod,coordinates,country,sourcename,sourcetype,mobile
6,"{utc=2017-08-11T05:00:00.000Z, local=2017-08-1...",pm25,McMillan NCORE,10.0,µg/m³,WASHINGTON,"[{name=US EPA AirNow, url=http://www.airnow.go...","{unit=hours, value=1.0}","{latitude=38.92185, longitude=-77.013176}",US,AirNow,government,False
10,"{utc=2017-08-11T05:00:00.000Z, local=2017-08-1...",pm25,Weiser Neph,19.9,µg/m³,WASHINGTON,"[{name=US EPA AirNow, url=http://www.airnow.go...","{unit=hours, value=1.0}","{latitude=44.261696, longitude=-116.97917}",US,AirNow,government,False
14,"{utc=2017-08-11T05:00:00.000Z, local=2017-08-1...",pm25,Sipayik-ME,6.5,µg/m³,WASHINGTON,"[{name=US EPA AirNow, url=http://www.airnow.go...","{unit=hours, value=1.0}","{latitude=44.9636, longitude=-67.0608}",US,AirNow,government,False
19,"{utc=2017-08-11T05:00:00.000Z, local=2017-08-1...",pm25,Rockville,14.0,µg/m³,Washington-Arlington-Alexandria,"[{name=US EPA AirNow, url=http://www.airnow.go...","{unit=hours, value=1.0}","{latitude=39.1144, longitude=-77.1069}",US,AirNow,government,False
22,"{utc=2017-08-11T05:00:00.000Z, local=2017-08-1...",pm25,HU-Beltsville,8.0,µg/m³,Washington-Arlington-Alexandria,"[{name=US EPA AirNow, url=http://www.airnow.go...","{unit=hours, value=1.0}","{latitude=39.0553, longitude=-76.8783}",US,AirNow,government,False


In [5]:
def getDateTime(row):
    json_s=row.date.replace("=", ":\"").replace("}","\"}").replace(".000Z",".000Z\"").replace('utc','"utc"').replace('local','"local"')
    #print(json_s)
    date_json = json.loads(json_s)
    return "{} UTC".format(date_json["utc"].replace("T", " ").replace(".000Z", ""))

def getX(row):
    geo_json = json.loads(row.coordinates.replace("=", ":").replace('longitude','"longitude"').replace('latitude','"latitude"'))
    #print(geo_json)
    return geo_json["latitude"]
    
def getY(row):
    geo_json = json.loads(row.coordinates.replace("=", ":").replace('longitude','"longitude"').replace('latitude','"latitude"'))
    return geo_json["longitude"]

df_PM25['datetime']= df_PM25.apply(lambda row: getDateTime(row), axis=1)
df_PM25['x']= df_PM25.apply(lambda row: getX(row), axis=1)
df_PM25['y']= df_PM25.apply(lambda row: getY(row), axis=1)
df_PM25['pm25']=df_PM25['value']
df_PM25['station_id']=df_PM25['attribution']
df_PM25['sensor']='ND'
df_PM25['type']=df_PM25['sourcename']

df_PM25.head()

/home/julien/.local/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
/home/julien/.local/lib/python3.7/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/julien/.local/lib/python3.7/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the do

,date,parameter,location,value,unit,city,attribution,averagingperiod,coordinates,country,sourcename,sourcetype,mobile,datetime,x,y,pm25,station_id,sensor,type
6,"{utc=2017-08-11T05:00:00.000Z, local=2017-08-1...",pm25,McMillan NCORE,10.0,µg/m³,WASHINGTON,"[{name=US EPA AirNow, url=http://www.airnow.go...","{unit=hours, value=1.0}","{latitude=38.92185, longitude=-77.013176}",US,AirNow,government,False,2017-08-11 05:00:00 UTC,38.921850,-77.013176,10.0,"[{name=US EPA AirNow, url=http://www.airnow.go...",ND,AirNow
10,"{utc=2017-08-11T05:00:00.000Z, local=2017-08-1...",pm25,Weiser Neph,19.9,µg/m³,WASHINGTON,"[{name=US EPA AirNow, url=http://www.airnow.go...","{unit=hours, value=1.0}","{latitude=44.261696, longitude=-116.97917}",US,AirNow,government,False,2017-08-11 05:00:00 UTC,44.261696,-116.979170,19.9,"[{name=US EPA AirNow, url=http://www.airnow.go...",ND,AirNow
14,"{utc=2017-08-11T05:00:00.000Z, local=2017-08-1...",pm25,Sipayik-ME,6.5,µg/m³,WASHINGTON,"[{name=US EPA AirNow, url=http://www.airnow.go...","{unit=hours, value=1.0}","{latitude=44.9636, longitude=-67.0608}",US,AirNow,government,False,2017-08-11 05:00:00 UTC,44.963600,-67.060800,6.5,"[{name=US EPA AirNow, url=http://www.airnow.go...",ND,AirNow
19,"{utc=2017-08-11T05:00:00.000Z, local=2017-08-1...",pm25,Rockville,14.0,µg/m³,Washington-Arlington-Alexandria,"[{name=US EPA AirNow, url=http://www.airnow.go...","{unit=hours, value=1.0}","{latitude=39.1144, longitude=-77.1069}",US,AirNow,government,False,2017-08-11 05:00:00 UTC,39.114400,-77.106900,14.0,"[{name=US EPA AirNow, url=http://www.airnow.go...",ND,AirNow
22,"{utc=2017-08-11T05:00:00.000Z, local=2017-08-1...",pm25,HU-Beltsville,8.0,µg/m³,Washington-Arlington-Alexandria,"[{name=US EPA AirNow, url=http://www.airnow.go...","{unit=hours, value=1.0}","{latitude=39.0553, longitude=-76.8783}",US,AirNow,government,False,2017-08-11 05:00:00 UTC,39.055300,-76.878300,8.0,"[{name=US EPA AirNow, url=http://www.airnow.go...",ND,AirNow


Filter here the ones inside DC metro area

Using the previously parsed X and Y from coordinates latitudes and longitudes

In [6]:
#same as worldpop 
#@todo x and y were inverted

buffer_deg=0.0027
ymax_MetroArea = -76.3881880661718 + buffer_deg
xmax_MetroArea = 39.5284754397557 + buffer_deg
ymin_MetroArea = -77.7356609923511 - buffer_deg
xmin_MetroArea = 38.3969644315365 - buffer_deg
    
geoFilter = (
    (df_PM25["x"] > xmin_MetroArea) &
    (df_PM25["x"] < xmax_MetroArea) &
    (df_PM25["y"] > ymin_MetroArea) &
    (df_PM25["y"] < ymax_MetroArea) 
)

len(df_PM25[geoFilter])

138727

In [7]:
df_PM25 = df_PM25[geoFilter]
#df_PM25[cols].head()
len(df_PM25)

138727

## For v1 we keep only 2019 records

In [8]:
dateFilter = df_PM25.datetime.str.startswith("2019")
df_PM25 = df_PM25[dateFilter]
#df_PM25[cols].head()
len(df_PM25)

47570

### Save record to bigtable along purpleair ones keeping only same columns

In [9]:
cols =["datetime", "pm25", "type", "sensor", "station_id", "x", "y"]


df_bigtable = pd.read_csv("bigtable.csv")
nb = len(df_bigtable)
df_bigtable = df_bigtable.append(df_PM25[cols])
print("{} records are now {}".format(nb, len(df_bigtable)))
df_bigtable.type.describe()

105054 records are now 152624


count        152624
unique            2
top       PurpleAir
freq         105054
Name: type, dtype: object

In [10]:
df_bigtable.to_csv("bigtable.csv")

# WorldPop data extract and wrangling

## worlpop data extract 

Done by python script.

Data is downloaded using python script using provided API 

Data is stored as tiff locally but after clipping it using DC metro area geo extent (because full US tiff is above 10Gb by tiff)

## worldpop data wrangling

Using the python script and GDAL OGR, will find for each entry in CSV file the corresponding value from geotiff

Using location

Store it in new csv columns 

Using and running python script with arfuments as follow:

In [11]:
%run worldpop/load-wp.py --csv bigtable.csv --folder worldpop/GIS

## Data wrangling make date cyclic

Python script that will transform datetime feature into 2 cyclic values :

### One for the time of the day

Traffic is known to have an impact to PM25 pollution and hour of the day is related to traffic importance.

### If this is a weekday or not

Same remark about traffic

### A second one for the day in the year

To challenge and check if seasonality is correlated winter/sumer ?


Using and running python script with arfuments as follow:

In [12]:
%run cyclic/build-cyclic.py --csv bigtable.csv --column datetime

## Weather / meteo data extract

### weather data extraction made manually and storde locally

### Data wrangling done here below

In [13]:
df = pd.read_csv("bigtable.csv")
df['time'] = pd.to_datetime(df["datetime"],format="%Y-%m-%d %H:%M:%S %Z")
df['date'] = pd.Series(df.time).dt.strftime("%Y-%m-%d")
df['hour'] = pd.Series(df.time).dt.strftime("%H")

print("nb ground station measurement : {}".format(len(df)))


#2019-01-01T02:40:00
df_meteo = pd.read_csv("weather/weather.csv")
df_meteo['time'] = pd.to_datetime(df_meteo["DATE"],format="%Y-%m-%dT%H:%M:%S")
df_meteo['date'] = pd.Series(df_meteo.time).dt.strftime("%Y-%m-%d")
df_meteo['hour'] = pd.Series(df_meteo.time).dt.strftime("%H")
df_meteo['datehour'] = pd.Series(df_meteo.time).dt.strftime("%Y-%m-%d-%H")

print("nb meteo : {}".format(len(df_meteo)))

df_meteo.head()

    

nb ground station measurement : 152624
nb meteo : 14449


,DATE,Temperature,Wind,Dew,Sky,Visibility,ATM,Wind-Dir,Wind-Rate,sin_wind,cos_wind,time,date,hour,datehour
0,2019-01-01T00:00:00,8.91,"999,9,C,0000,1",7.81,99999,6000,1016.51,999,0.0,NaN,NaN,2019-01-01 00:00:00,2019-01-01,00,2019-01-01-00
1,2019-01-01T00:11:00,8.95,"999,9,C,0000,5",7.85,152,4023,9999.99,999,0.0,NaN,NaN,2019-01-01 00:11:00,2019-01-01,00,2019-01-01-00
2,2019-01-01T00:17:00,8.95,"999,9,C,0000,5",7.85,122,2414,9999.99,999,0.0,NaN,NaN,2019-01-01 00:17:00,2019-01-01,00,2019-01-01-00
3,2019-01-01T00:22:00,8.35,"999,9,C,0000,5",6.75,122,1207,9999.99,999,0.0,NaN,NaN,2019-01-01 00:22:00,2019-01-01,00,2019-01-01-00
4,2019-01-01T00:47:00,8.05,"999,9,C,0000,5",7.05,122,1609,9999.99,999,0.0,NaN,NaN,2019-01-01 00:47:00,2019-01-01,00,2019-01-01-00


In [14]:
df_meteo=df_meteo.groupby(['datehour']).first().reset_index()

print("nb meteo unique hourly : {}".format(len(df_meteo)))

df_meteo.head()


nb meteo unique hourly : 8759


,datehour,DATE,Temperature,Wind,Dew,Sky,Visibility,ATM,Wind-Dir,Wind-Rate,sin_wind,cos_wind,time,date,hour
0,2019-01-01-00,2019-01-01T00:00:00,8.91,"999,9,C,0000,1",7.81,99999,6000,1016.51,999,0.0,NaN,NaN,2019-01-01 00:00:00,2019-01-01,00
1,2019-01-01-01,2019-01-01T01:14:00,8.95,"999,9,C,0000,5",7.85,91,3219,9999.99,999,0.0,1.224647e-16,-1.000000,2019-01-01 01:14:00,2019-01-01,01
2,2019-01-01-02,2019-01-01T02:10:00,9.45,"200,5,N,0015,5",8.35,91,3219,9999.99,200,1.5,-3.420201e-01,-0.939693,2019-01-01 02:10:00,2019-01-01,02
3,2019-01-01-03,2019-01-01T03:00:00,10.01,"220,1,N,0031,1",8.91,99999,4800,1012.71,220,3.1,-6.427876e-01,-0.766044,2019-01-01 03:00:00,2019-01-01,03
4,2019-01-01-04,2019-01-01T04:52:00,10.65,"200,5,N,0026,5",9.45,91,1609,1010.65,200,2.6,-3.420201e-01,-0.939693,2019-01-01 04:52:00,2019-01-01,04


In [15]:
moments = df[['date', 'hour']].apply(tuple, axis=1).tolist()
moments = list(set(moments))
print("nb moments for stations measures : {}".format(len(moments)))


nb moments for stations measures : 8784


In [16]:
new_df = pd.merge(left=df, right=df_meteo, how='left', 
                  left_on=['date','hour'], right_on=['date','hour'])

print("nb ground station measurement after : {}".format(len(new_df)))
print("nb ground station measurement before : {}".format(len(df)))


new_df = new_df.drop(["time_x", "hour", "time_y"], axis=1)

print("***")

nb ground station measurement after : 152624
nb ground station measurement before : 152624
***


In [17]:
new_df.head()

,Unnamed: 0,datetime,pm25,type,sensor,station_id,x,y,Population,Dist-MRoads,...,Temperature,Wind,Dew,Sky,Visibility,ATM,Wind-Dir,Wind-Rate,sin_wind,cos_wind
0,0,2019-05-08 15:00:00 UTC,52.33,PurpleAir,B,14th & S ST NW B (undefined) (38.913805 -77.0...,38.913805,-77.03275,54.04969,0.072,...,21.11,"050,1,N,0036,1",16.11,99999.0,16000.0,1021.81,50.0,3.6,0.766044,0.642788
1,1,2019-05-08 16:00:00 UTC,0.17,PurpleAir,B,14th & S ST NW B (undefined) (38.913805 -77.0...,38.913805,-77.03275,54.04969,0.072,...,22.85,"030,5,N,0041,5",16.75,762.0,16093.0,1021.35,30.0,4.1,0.500000,0.866025
2,2,2019-05-08 15:00:00 UTC,52.33,PurpleAir,B,14th & S ST NW B (undefined) (38.913805 -77.0...,38.913805,-77.03275,54.04969,0.072,...,21.11,"050,1,N,0036,1",16.11,99999.0,16000.0,1021.81,50.0,3.6,0.766044,0.642788
3,3,2019-05-08 16:00:00 UTC,0.17,PurpleAir,B,14th & S ST NW B (undefined) (38.913805 -77.0...,38.913805,-77.03275,54.04969,0.072,...,22.85,"030,5,N,0041,5",16.75,762.0,16093.0,1021.35,30.0,4.1,0.500000,0.866025
4,4,2019-05-08 17:00:00 UTC,NaN,PurpleAir,B,14th & S ST NW B (undefined) (38.913805 -77.0...,38.913805,-77.03275,54.04969,0.072,...,22.85,"060,5,N,0057,5",16.75,1006.0,16093.0,1021.15,60.0,5.7,0.866025,0.500000


In [18]:
df =new_df

df["Temperature"]=pd.Series(df.Temperature).replace('\+', '', regex=True).replace(',', '.', regex=True).apply(pd.to_numeric, args=('coerce',))
df["Dew"]=pd.Series(df.Dew).replace('\+', '', regex=True).replace(',', '.', regex=True).apply(pd.to_numeric, args=('coerce',))
df["ATM"]=pd.Series(df.ATM).replace('\+', '', regex=True).replace(',', '.', regex=True).apply(pd.to_numeric, args=('coerce',))



In [19]:
df["station_id"].unique()

array(['14th & S ST NW  B (undefined) (38.913805 -77.03275) Primary 60_minute_average 01_01_2019 01_01_2020.csv',
       '13th and E St SE B (undefined) (38.882974 -76.988037) Primary 60_minute_average 01_01_2019 01_01_2020.csv',
       'Courthouse  (outside) (38.88812 -77.088094) Primary 60_minute_average 01_01_2019 01_01_2020.csv',
       'Cheverly (outside) (38.921633 -76.921768) Primary 60_minute_average 01_01_2019 01_01_2020-1.csv',
       '13th and E St SE (outside) (38.882974 -76.988037) Primary 60_minute_average 01_01_2019 01_01_2020.csv',
       'V Street (outside) (38.918491 -77.037393) Primary 60_minute_average 01_01_2019 01_01_2020.csv',
       'Cheverly (outside) (38.921633 -76.921768) Primary 60_minute_average 01_01_2019 01_01_2020-9.csv',
       '14th & S ST NW  (outside) (38.913805 -77.03275) Primary 60_minute_average 01_01_2019 01_01_2020.csv',
       'arlington (outside) (38.900099 -77.081078) Primary 60_minute_average 01_01_2019 01_01_2020-1.csv',
       'McMillan 1 

In [20]:
# Simplify column names for stations
columns = {"Population":"population", "Dist-MRoads":"dist-mroads", "Dist-Setl":"dist-setl", "Dist-Coast":"dist-coast", "Dist-Forest":"dist-forest", "Slope":"slope", "Elevation":"elevation", "DATE":"date", "Temperature":"TEMP", "Wind":"WIND", "Dew":"DEW", "Sky":"SKY", "Visibility":"VIS"}
df = df.rename(columns=columns)
df = df.replace('14th & S ST NW  B (undefined) (38.913805 -77.03275) Primary 60_minute_average 01_01_2019 01_01_2020.csv', '14th & S ST NW B')
df = df.replace('14th & S ST NW  (outside) (38.913805 -77.03275) Primary 60_minute_average 01_01_2019 01_01_2020.csv', '14th & S ST NW A')
df = df.replace('V Street (outside) (38.918491 -77.037393) Primary 60_minute_average 01_01_2019 01_01_2020.csv', 'V Street')
df = df.replace('McMillan 1 (outside) (38.92185 -77.013271) Primary 60_minute_average 01_01_2019 01_01_2020.csv', 'McMillan 1')
df = df.replace('Courthouse  (outside) (38.88812 -77.088094) Primary 60_minute_average 01_01_2019 01_01_2020.csv', 'Courthouse')
df = df.replace('Cheverly (outside) (38.921633 -76.921768) Primary 60_minute_average 01_01_2019 01_01_2020-9.csv', 'Cheverly 9')
df = df.replace('Cheverly (outside) (38.921633 -76.921768) Primary 60_minute_average 01_01_2019 01_01_2020-1.csv', 'Cheverly 1')
df = df.replace('arlington (outside) (38.900099 -77.081078) Primary 60_minute_average 01_01_2019 01_01_2020-1.csv', 'Arlington')
df = df.replace('13th and E St SE B (undefined) (38.882974 -76.988037) Primary 60_minute_average 01_01_2019 01_01_2020.csv', '13th & E ST SE B')
df = df.replace('13th and E St SE (outside) (38.882974 -76.988037) Primary 60_minute_average 01_01_2019 01_01_2020.csv', '13th & E ST SE A')
df = df.replace('[{name=US EPA AirNow, url=http://www.airnow.gov/}, {name=Maryland Department of the Environment, url=null}]', 'MA_EPA')
df = df.replace('[{name=US EPA AirNow, url=http://www.airnow.gov/}, {name=Virginia Dept. of Environmental Quality, url=null}]', 'VA_EPA')
df = df.replace('[{name=US EPA AirNow, url=http://www.airnow.gov/}, {name=District of Columbia - Department of Energy and Environment, url=null}]', 'DC_EPA')



df.head()


,Unnamed: 0,datetime,pm25,type,sensor,station_id,x,y,population,dist-mroads,...,TEMP,WIND,DEW,SKY,VIS,ATM,Wind-Dir,Wind-Rate,sin_wind,cos_wind
0,0,2019-05-08 15:00:00 UTC,52.33,PurpleAir,B,14th & S ST NW B,38.913805,-77.03275,54.04969,0.072,...,21.11,"050,1,N,0036,1",16.11,99999.0,16000.0,1021.81,50.0,3.6,0.766044,0.642788
1,1,2019-05-08 16:00:00 UTC,0.17,PurpleAir,B,14th & S ST NW B,38.913805,-77.03275,54.04969,0.072,...,22.85,"030,5,N,0041,5",16.75,762.0,16093.0,1021.35,30.0,4.1,0.500000,0.866025
2,2,2019-05-08 15:00:00 UTC,52.33,PurpleAir,B,14th & S ST NW B,38.913805,-77.03275,54.04969,0.072,...,21.11,"050,1,N,0036,1",16.11,99999.0,16000.0,1021.81,50.0,3.6,0.766044,0.642788
3,3,2019-05-08 16:00:00 UTC,0.17,PurpleAir,B,14th & S ST NW B,38.913805,-77.03275,54.04969,0.072,...,22.85,"030,5,N,0041,5",16.75,762.0,16093.0,1021.35,30.0,4.1,0.500000,0.866025
4,4,2019-05-08 17:00:00 UTC,NaN,PurpleAir,B,14th & S ST NW B,38.913805,-77.03275,54.04969,0.072,...,22.85,"060,5,N,0057,5",16.75,1006.0,16093.0,1021.15,60.0,5.7,0.866025,0.500000


In [21]:
#Basic data wrangling now performed in notebook in data/weather/app.ipynb

df = df.drop(columns=['WIND'])
#Explain and transform data to numeric vetor or absolute value

# The rate of horizontal travel of air past a fixed point.
# MIN: 0000 MAX: 0900 UNITS: meters per second SCALING FACTOR: 10 
#data = df['WIND'].str.split(pat=",", expand=True)
#df['WIND'] = data[3].str.lstrip('0')

# The height above ground level (AGL) of the lowest cloud or obscuring phenomena layer aloft with 5/8 or more summation total sky cover, which may be predominantly opaque, or the vertical visibility into a surface-based obstruction. Unlimited = 22000.
# MIN: 00000 MAX: 22000 UNITS: Meters
#data = df['SKY'].str.split(pat=",", expand=True)
#df['SKY'] = data[0].str.lstrip('0')

# The horizontal distance at which an object can be seen and identified.
# MIN: 000000 MAX: 160000 UNITS: Meters
#data = df['VIS'].str.split(pat=",", expand=True)
#df['VIS'] = data[0].str.lstrip('0')



## create categories US EPA PM2.5 AQI

https://www.epa.gov/sites/production/files/2014-05/documents/zell-aqi.pdf

![AQI](USEPAPM25AQI.png "US EPA PM2.5 AQI")

First remove "blank and extreme outliers" to avoid noise data

**Only keep range 1 to 500**

Important note our categories are not the US EPA PM2.5 AQI but they are still usefull to use them to categorize  real time classification. We use them for instantaneous measures in AQI_class, the real AQI for information only is stored in AQI column (24hr mean).


In [22]:
AQI = {
    "Good" : {
        "min_AQI" : 0,
        "max_AQI" : 50,
        "min_pm25" : 0,
        "max_pm25" : 15.4,
    },
    "Moderate" : {
        "min_AQI" : 50,
        "max_AQI" : 100,
        "min_pm25" : 15.4,
        "max_pm25" : 40.4,
    },
    "USG" : {
        "min_AQI" : 100,
        "max_AQI" : 150,
        "min_pm25" : 40.4,
        "max_pm25" : 65.4,
    },
    "Unhealthy" : {
        "min_AQI" : 150,
        "max_AQI" : 200,
        "min_pm25" : 65.4,
        "max_pm25" : 150.4,
    },
    "Very unhealthy" : {
        "min_AQI" : 200,
        "max_AQI" : 300,
        "min_pm25" : 150.4,
        "max_pm25" : 250.4,
    },
    "Hazardous" : {
        "min_AQI" : 300,
        "max_AQI" : 500,
        "min_pm25" : 250.4,
        "max_pm25" : 500.4,
    },
}

def getInstentaneousAQIClass(pm25):
    """ Return AQI classification for a single instantaneous value"""
    return getAQIClass(pm25)


def get24hourAQIClass(pm25_list):
    """ Return real AQI for a pm25 list assuming the measurements are done homogeneously"""
    return getAQIClass(sum(lst) / len(lst))


def getAQIClass(pm25):
    """ Return AQI classification name and AQI value for a pm25 measurement"""
    for className, AQI_Spec in AQI.items():
        if pm25<AQI_Spec["max_pm25"]:
            ratio = (pm25 - AQI_Spec["min_pm25"]) / (AQI_Spec["max_pm25"] - AQI_Spec["min_pm25"])
            return [className, (AQI_Spec["min_AQI"] + ratio * (AQI_Spec["max_AQI"] - AQI_Spec["min_AQI"]))]
    return["Not found", NaN]

print(getInstentaneousAQIClass(34.0))

['Moderate', 87.2]


In [23]:
df_filter = (df.pm25 > 1) & (df.pm25 < 500)
print("{} records - {} outliers = {} valid records".format(len(df), len(df) - len(df[df_filter]), len(df[df_filter])))
df_clean = df[df_filter]
df_clean = df_clean.drop_duplicates(subset=['station_id', 'datetime', 'sensor', 'x', 'y'])
df_clean=df_clean.drop([ "date", "datehour", "24hr", "Unnamed: 0"], axis=1)
print(len(df_clean))
df_clean.columns

152624 records - 5993 outliers = 146631 valid records
105398


Index(['datetime', 'pm25', 'type', 'sensor', 'station_id', 'x', 'y',
       'population', 'dist-mroads', 'dist-setl', 'dist-coast', 'dist-forest',
       'slope', 'elevation', 'dayofweek', 'sin_day', 'cos_day', 'sin_year',
       'cos_year', 'TEMP', 'DEW', 'SKY', 'VIS', 'ATM', 'Wind-Dir', 'Wind-Rate',
       'sin_wind', 'cos_wind'],
      dtype='object')

In [24]:
#@todo COMPUTE real PM25 (averge of last 24hours)

#help of
#https://stackoverflow.com/questions/36969174/pandas-average-value-for-the-past-n-days
#https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.rolling.html
#https://stackoverflow.com/questions/56942713/calculating-rolling-mean-of-15-minutes-8-hours-and-24-hours-using-python

df_clean["datetime"] = pd.to_datetime(df_clean["datetime"])
df_clean_st = df_clean.groupby('station_id').apply(lambda x: x.set_index('datetime').resample('24H').first())
df_AQI = df_clean_st.groupby(level=0)['pm25'].apply(lambda x: x.shift().rolling(min_periods=1,window=1).mean()).reset_index(name='PM25_average24hours')

In [25]:
df_clean["AQI_class"] = df_clean["pm25"]
df_clean["AQI_VALUE"] = df_clean["pm25"]
df_clean["AQI_class"] = df_clean["AQI_class"].apply(lambda x: getInstentaneousAQIClass(x)[0])
df_clean["AQI_VALUE"] = df_clean["AQI_VALUE"].apply(lambda x: getInstentaneousAQIClass(x)[1])
df_clean.head()

,datetime,pm25,type,sensor,station_id,x,y,population,dist-mroads,dist-setl,...,DEW,SKY,VIS,ATM,Wind-Dir,Wind-Rate,sin_wind,cos_wind,AQI_class,AQI_VALUE
0,2019-05-08 15:00:00+00:00,52.33,PurpleAir,B,14th & S ST NW B,38.913805,-77.03275,54.04969,0.072,-1.541,...,16.11,99999.0,16000.0,1021.81,50.0,3.6,0.766044,6.427876e-01,USG,123.860000
26,2019-05-09 15:00:00+00:00,43.76,PurpleAir,B,14th & S ST NW B,38.913805,-77.03275,54.04969,0.072,-1.541,...,16.71,99999.0,16000.0,1021.31,90.0,3.6,1.000000,6.123234e-17,USG,106.720000
33,2019-05-16 16:00:00+00:00,11.41,PurpleAir,B,14th & S ST NW B,38.913805,-77.03275,54.04969,0.072,-1.541,...,10.65,22000.0,16093.0,1013.15,330.0,3.6,-0.500000,8.660254e-01,Good,37.045455
34,2019-05-16 17:00:00+00:00,10.24,PurpleAir,B,14th & S ST NW B,38.913805,-77.03275,54.04969,0.072,-1.541,...,10.65,22000.0,16093.0,1012.65,320.0,5.7,-0.642788,7.660444e-01,Good,33.246753
35,2019-05-16 18:00:00+00:00,9.45,PurpleAir,B,14th & S ST NW B,38.913805,-77.03275,54.04969,0.072,-1.541,...,10.61,22000.0,16000.0,1012.61,320.0,5.7,-0.642788,7.660444e-01,Good,30.681818


In [26]:
#df.to_csv("bigtable.csv", index=False)
df_clean.to_csv("bigtable.csv", index=False)